##Install Pre req from Kaggle

Download the required datasets from the Kaggle repository.

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
cd /root/.kaggle

/root/.kaggle


In [ ]:
!pwd
!ls

/root/.kaggle


Upload the json file for Kaggle authorization

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cd /content

/content


In [ ]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 27.3MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 21.1MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.3MB/s]


#Import required libs

In [ ]:
!pip install autocorrect

     |████████████████████████████████| 624kB 8.2MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.3.0-cp36-none-any.whl size=621588 sha256=5a32ee73785698eece3d3c6ec44cdcd807b6ffb9b738324d8294bf287597f282
  Stored in directory: /root/.cache/pip/wheels/cc/1c/30/6b0199afbd20eef5959f5eaa0ead86aeef84391740482b2279
Successfully built autocorrect


In [ ]:
import numpy as np
import pandas as pd


import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from autocorrect import Speller


    


import json
import os


pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

#Loading the data

In [ ]:
Tweets = pd.read_csv("/content/train.csv", encoding="utf-8")

#Helper functions

Some helper functions for preprocessing the tweets are defined here

Load the dictionaries from the GitHub repositiory to process contracted words and words with repeated characters.

**Contracted words**




Eg  I've  = I have

**Repeated characters words**




Eg  goooaaalll  = goal

coooool     = cool
    

In [ ]:
!wget https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/contractions_dict.json
!wget https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/repetitions_dict.json
!wget https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/emoticons_dict.json
!wget https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/html_tags_dict.json
!wget https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/chat_words_dict.json

--2021-02-17 21:52:06--  https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/contractions_dict.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2825 (2.8K) [text/plain]
Saving to: ‘contractions_dict.json.1’

contractions_dict.j 100%[===================>]   2.76K  --.-KB/s    in 0s      

2021-02-17 21:52:06 (51.9 MB/s) - ‘contractions_dict.json.1’ saved [2825/2825]

--2021-02-17 21:52:06--  https://raw.githubusercontent.com/Mikail1010/AI-ML-contests/main/NLP%20for%20Disaster%20Tweets/Helper%20dicts/repetitions_dict.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

In [ ]:
with open('contractions_dict.json', 'r') as fp:
    contractions_dict = json.load(fp)
with open('repetitions_dict.json', 'r') as fp:
    repetitions_dict = json.load(fp)
with open('emoticons_dict.json', 'r') as fp:
    emoticons_dict = json.load(fp)
with open('html_tags_dict.json', 'r') as fp:
    html_tags_dict = json.load(fp)
html_tags_list = (html_tags_dict['HTML tags']).split(',')
with open('chat_words_dict.json', 'r') as fp:
    chat_words_dict = json.load(fp)

print('Contracted Words =', len(contractions_dict), 'Repetition Letter Words =',len(repetitions_dict), 'Emoticons =',len(emoticons_dict),
      'html_tags =', len(html_tags_list), 'Chat Words =', len(chat_words_dict))

Contracted Words = 106 Repetition Letter Words = 13 Emoticons = 19 html_tags = 8 Chat Words = 66


Helper functions for basic text processing, tweet processing, extracting features and tokenization

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
stopwords.words('english')
tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
chat_words = re.compile('\\b' + '\\b|\\b'.join(list(chat_words_dict.keys())))
spell = Speller(lang='en')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def BasicTextProcessing(text, remove_url, lower_text_case,
                        remove_nos, remove_punctutation, remove_html_tags, convert_html_text,
                        chat_lingo_replace
                        ):
  # Removing line breaks, html tags, certain unicode symbols
  # To remove url, nos and punctutation and convert to lower case if necessary
  if remove_url:
    text = re.sub('https?://\S+|www\.\S+', '', text)

  if lower_text_case:
    text = text.lower()
  if chat_lingo_replace:
    for keys,vals in chat_words_dict.items():
      text = re.sub('\\b' + keys + '\\b', vals, text)

  text = re.sub('\\n', '', text)
  text = re.sub('ûª', "'", text)
  text = re.sub('&amp;', 'and', text)
  text = re.sub('&lt;', '', text)
  text = re.sub('&gt;', '', text)
  if remove_nos:
    text = re.sub('\d+', '', text)
  if remove_html_tags:
    for tag in html_tags_list:
      text = re.sub(tag, '', text)
  if convert_html_text:
      text = BeautifulSoup(text, "lxml").text

  return text

def RemovePunctutation(text):
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  return text

def SpellCheck(text):
    corrected_text = []
    misspelled_text = text.split()
    for word in text.split():
        corrected_text.append(spell(word))
    return " ".join(corrected_text)
        
#   text = re.sub('\[.*?\]', '', text)
#   text = re.sub('<.*?>+', '', text)
#   text = re.sub('\w*\d\w*', '', text)

In [ ]:
# To extract URLs as a seperate feature
def ExtractURL(text):
  match = re.findall('https?://\S+|www\.\S+', text)
  if match == []:
    url = np.nan
  else:
    url = match
  return url

# To extract Hashtags as a seperate feature
def Extracthashtag(text):
  match = re.findall('#\S+', text)
  if match == []:
    hashtag = np.nan
  else:
    hashtag = match
    for idx, i in enumerate(hashtag):
      hashtag[idx] = re.sub('[%s]' % re.escape(string.punctuation), '', i)
  return hashtag

# To extract Mentioned Names as a seperate feature
def ExtractMentioned(text):
  match = re.findall('@\S+', text)
  if match == []:
    mentioned = np.nan
  else:
    mentioned = match
    for idx, i in enumerate(mentioned):
      mentioned[idx] = re.sub('[@]' , '', i)
  return mentioned

# To extract Emojis as a seperate feature
def ExtractEmoji(text):
  match = re.findall('@\S+', text)
  if match == []:
    emoji = np.nan
  else:
    emoji = match
  return emoji

# To extract Emoticons as a seperate feature
def ExtractEmoticons(text):
  match = []
  for key, value in emoticons_dict.items():
    if key in text:
      match.append(key)
  if match == []:
    emoticons = np.nan
  else: 
    emoticons = [emoticons_dict[i] for i in match]
  return emoticons

def chat_words_dict_E(text):
  match = re.findall(chat_words, text)
  # print(match)
  if match == []:
    chat = np.nan
  else: 
    chat = [chat_words_dict[i] for i in match]
  return chat

In [ ]:
# To perfrom tweets specific text processing
def FixCommonTweetErrors(text, remove_hashtag, remove_mentioned, remove_emoji, remove_emoticons):
  # To expand contracted words
  def Expand_Contractions(text):
    for keys,vals in contractions_dict.items():
      text = re.sub(keys, vals, text)
    return text
  # To contract words with repeated letters
  def Contract_Repetitions(text):
    for keys,vals in repetitions_dict.items():
      text = re.sub(keys, vals, text)
    return text
  def remove_emoticons(text):
    for keys,vals in emoticons_dict.items():
      text = re.sub(keys, vals, text)
    return text
  # To remove hashtaged words from text if necessary
  if remove_hashtag:
    text = re.sub('#\S+', '', text)
  if remove_mentioned:
    text = re.sub('@\S+', '', text)
  if remove_emoji:
    text = emoji_pattern.sub(r'', text)
  if remove_emoticons:
    text = Contract_Repetitions(text)


  text = Contract_Repetitions(text)
  text = Expand_Contractions(text)
  return text

In [ ]:
# To remove stopwords if necessary
def RemoveStopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words 

# To combine tokenized words if necessary
def CombineText(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

def StemWords(text, stemmer):
  text = " ".join([stemmer.stem(word) for word in text.split()])
  return text

def LemmatizeWords(text,lemmatizer):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

#Pipeline

Pipleline to perfrom all preprocessing functions.

Non - required for preprocessing can be made false

**Default values**

remove_url=True, extract_url=False, remove_punctutation=True, extract_hashtag=False, 
                  lower_text_case=True, remove_nos=True, fix_common_tweet_errors=False,
                  remove_hashtag=False, remove_stopwords=False, combine_text=False,
                  extract_mentioned = True, remove_mentioned=False, stem_words=False,
                  lemmatize_words=False

In [ ]:
def CleanTextData(Data, remove_url=True, extract_url=False, remove_punctutation=True, extract_hashtag=False, 
                  lower_text_case=True, remove_nos=True, fix_common_tweet_errors=False,
                  remove_hashtag=False, remove_stopwords=False, combine_text=False,
                  extract_mentioned = True, remove_mentioned=False, stem_words=False,
                  lemmatize_words=False, remove_emoji=True, extract_emoji=False,
                  extract_emoticons=False, remove_emoticons=True, remove_html_tags=True,
                  convert_html_text=False, chat_lingo_replace=True, spell_check=False
                  ):
  if convert_html_text:
    from bs4 import BeautifulSoup

  
  Data = Data.assign(text_pre_pro = lambda x: (x['text'].apply(lambda x:BasicTextProcessing(x, remove_url, lower_text_case,
                        remove_nos, remove_punctutation, remove_html_tags, convert_html_text,
                        chat_lingo_replace))))

  if fix_common_tweet_errors:
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:FixCommonTweetErrors(x, remove_hashtag, remove_mentioned, remove_emoji, remove_emoticons)) 

  if extract_hashtag:
    Data = Data.assign(hashtag=lambda x: (x['text'].apply(lambda x:Extracthashtag(x))))

  if extract_url:
    Data = Data.assign(url=lambda x: (x['text'].apply(lambda x:ExtractURL(x))))

  if extract_mentioned:
    Data = Data.assign(mentioned=lambda x: (x['text'].apply(lambda x:ExtractMentioned(x))))

  if extract_emoji:
    Data = Data.assign(emoji=lambda x: (x['text'].apply(lambda x:ExtractEmoji(x))))

  if extract_emoticons:
    Data = Data.assign(emoticons=lambda x: (x['text'].apply(lambda x:ExtractEmoticons(x))))
  

  if remove_punctutation:
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:RemovePunctutation(x))

  if spell_check:
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:SpellCheck(x))
 
  if remove_stopwords:
    LoadStopwords()
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:RemoveStopwords(x))

  if combine_text:
    Data['tokenized_processed'] = Data['text_pre_pro'].apply(lambda x:CombineText(x))

  if stem_words:
    stemmer = PorterStemmer()
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:StemWords(x, stemmer))

  if lemmatize_words:    
    lemmatizer = WordNetLemmatizer()  
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:LemmatizeWords(x, lemmatizer))
  Data = Data.assign(tokenized_processed=lambda x: (x['text_pre_pro'].apply(lambda x:tokenizer.tokenize(x))))
 

  return Data

#Processing the data

In [ ]:
Tweets_PreProcessed = CleanTextData(Tweets, extract_hashtag=True, 
                                        fix_common_tweet_errors=True,
                                        extract_url = True,
                                        remove_mentioned=True,
                                        )

# Some ids of tweets which can be used to check if the prrocssing had the inteded
# effect on the tweets
# listofid = [28, 109, 158, 220, 232, 409, 1722]

# for i in listofid:
#   print(Tweets_PreProcessed.loc[Tweets_PreProcessed['id'] == i, ['text', 'text_pre_pro']])

Tweets_PreProcessed.head(4)

,id,keyword,location,text,target,text_pre_pro,hashtag,url,mentioned,tokenized_processed
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,our deeds are the reason of this earthquake may allah forgive us all,[earthquake],NaN,NaN,"[our, deeds, are, the, reason, of, this, earthquake, may, allah, forgive, us, all]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,NaN,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,NaN,NaN,NaN,"[all, residents, asked, to, shelter, in, place, are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,people receive wildfires evacuation orders in california,[wildfires],NaN,NaN,"[people, receive, wildfires, evacuation, orders, in, california]"


In [ ]:
# Tweets_PreProcessed['mentioned'][Tweets_PreProcessed['mentioned'].notna()]

In [ ]:
Tweets_PreProcessed.to_csv('train_processed.csv')

Reset the variables if necessary

In [ ]:
# %reset

#Sub Program to write new values to dicts

In [ ]:
contractions_dict.items()

dict_items([("ain't", 'are not'), ("'s", ' is'), ("aren't", 'are not'), ("can't", 'cannot'), ("can't've", 'cannot have'), ("'cause", 'because'), ("could've", 'could have'), ("couldn't", 'could not'), ("couldn't've", 'could not have'), ("didn't", 'did not'), ("doesn't", 'does not'), ("don't", 'do not'), ("hadn't", 'had not'), ("hadn't've", 'had not have'), ("hasn't", 'has not'), ("haven't", 'have not'), ("he'd", 'he would'), ("he'd've", 'he would have'), ("he'll", 'he will'), ("he'll've", 'he will have'), ("how'd", 'how did'), ("how'd'y", 'how do you'), ("how'll", 'how will'), ("i'd", 'i would'), ("i'd've", 'i would have'), ("i'll", 'i will'), ("i'll've", 'i will have'), ("i'm", 'i am'), ("i've", 'i have'), ("isn't", 'is not'), ("it'd", 'it would'), ("it'd've", 'it would have'), ("it'll", 'it will'), ("it'll've", 'it will have'), ("let's", 'let us'), ("ma'am", 'madam'), ("mayn't", 'may not'), ("might've", 'might have'), ("mightn't", 'might not'), ("mightn't've", 'might not have'), ("mus

Add any new values to the dicts

In [ ]:
contractions_dict['CONTRACTED WORD'] = 'EXPANDED WORD'
repetitions_dict['REPEATED LETTER WORD'] = 'CONTRACTED WORD'

Save the dict and use it for further preprocessing

In [ ]:
with open('contractions_dict.json', 'w') as fp:
    json.dump(contractions_dict, fp)
with open('repetitions_dict.json', 'w') as fp:
    json.dump(repetitions_dict, fp)

#Sub Program to find list of words with repeated leterrs (Eg: gooooaaalll)

Check the number of words with repeated letters in your dataset and add any new words to the dict if necessary 

In [ ]:
word_corpus = []
j = 0
for tweet in Tweets_PreProcessed['tokenized_processed']:
  for word in tweet:
    if word in word_corpus:
      j = j +1
    else:
      word_corpus.append(word)
print(len(word_corpus))

17118


In [ ]:
Consec_2_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 1 < len(word):
      if word[idx] == word[idx + 1]:
        Consec_2_letter_words.append(word)
print(len(Consec_2_letter_words))
print(Consec_2_letter_words)

4160
['deeds', 'allah', 'all', 'officers', 'school', 'flood', 'flooding', 'streets', 'hill', 'see', 'woods', 'happening', 'across', 'street', 'three', 'getting', 'flooded', 'gonna', 'arrived', 'summer', 'cool', 'skiing', 'week', 'bbcmtd', 'look', 'will', 'office', 'soooo', 'soooo', 'soooo', 'followers', 'better', 'setting', 'alexissanchez', 'happy', 'teammates', 'gunners', 'offices', 'voortrekker', 'voortrekker', 'kiss', 'filled', 'peeps', 'farewell', 'progressive', 'greetingsin', 'fallen', 'hood', 'buff', 'accident', 'impossible', 'nashvilletraffic', 'nashvilletraffic', 'traffic', 'caraccidentlawyer', 'speeding', 'teen', 'accidents', 'tee', 'curry', 'awareness', 'mooresville', 'mooresville', 'iredell', 'sleepjunkies', 'sleeping', 'pills', 'happen', 'cabrillo', 'hwymagellan', 'accidentwho', 'mayonnaise', 'horrible', 'finally', 'pissed', 'donnie', 'tell', 'been', 'ashville', 'crossed', 'naayf', 'chandanee', 'mma', 'rammed', 'eddy', 'willis', 'aashiqui', 'actress', 'aggarwal', 'suffield'

In [ ]:
Consec_3_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 2 < len(word):
      if word[idx] == word[idx + 1] == word[idx + 2]:
        Consec_3_letter_words.append(word)
print(len(Consec_3_letter_words))
print(Consec_3_letter_words)

196
['soooo', 'soooo', 'awwww', 'awwww', 'mhmmm', 'alexshipppp', 'alexshipppp', 'avysss', 'aiii', 'baaaack', 'baaaack', 'iii', 'xdojjjj', 'xdojjjj', 'omgbethersss', 'xxx', 'grrrr', 'grrrr', 'wwwbigbaldhead', 'zzzz', 'zzzz', 'lmfaoooo', 'lmfaoooo', 'deeeznvtzzz', 'deeeznvtzzz', 'oooureli', 'mmm', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'uhhhhh', 'uhhhhh', 'uhhhhh', 'ieee', 'rokiieee', 'loveyouuuu', 'loveyouuuu', 'sooo', 'michelleellle', 'emaaalay', 'crosssectarian', 'naaa', 'kwaaaaadead', 'kwaaaaadead', 'kwaaaaadead', 'ssshhheeesshh', 'ssshhheeesshh', 'ssshhheeesshh', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'shoook', 'frackfreeeu', 'stiiilo', 'ahhhhh', 'ahhhhh', 'ahhhhh', 'alllivesmatter', 'ohyayyyyay', 'ohyayyyyay', 'ayyy', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'ssssnell', 'ssssnell', 'www', 'iiii', 'iiii', 'riveeeeeer', 'riveeeeeer'

In [ ]:
Consec_4_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 3 < len(word):
      if word[idx] == word[idx + 1] == word[idx + 2] == word[idx + 3]:
        Consec_4_letter_words.append(word)
print(len(Consec_4_letter_words))
print(Consec_4_letter_words)

84
['soooo', 'awwww', 'alexshipppp', 'baaaack', 'xdojjjj', 'grrrr', 'zzzz', 'lmfaoooo', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'uhhhhh', 'uhhhhh', 'loveyouuuu', 'kwaaaaadead', 'kwaaaaadead', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'ahhhhh', 'ahhhhh', 'ohyayyyyay', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'ssssnell', 'iiii', 'riveeeeeer', 'riveeeeeer', 'riveeeeeer', 'milioooo', 'roomsgrrrr', 'alllll', 'alllll', 'nowwwwww', 'nowwwwww', 'nowwwwww', 'wompppp', 'selmoooooo', 'selmoooooo', 'selmoooooo', 'errrr', 'damnnnn', 'aannnnd', 'shidddd', 'ahhhh', 'omgggg', 'aaaa', 'onnnn', 'comeeeee', 'comeeeee', 'mxaaaa', 'llll', 'blaaaaaaa', 'blaaaaaaa', 'blaaaaaaa', 'blaaaaaaa', 'ayhhhhhdjjfjrjjrdjjeks', 'ayhhhhhdjjfjrjjrdjjeks', 'mochichiiiii', 'mochichiiiii', 'sheetingaaaaaand', 'sheetingaaaaaand', 'sheetingaaaaaand', 'maaaaan', 'maaaaan', 'ruddyyyyyy', 'ruddyyyyyy', 'ruddyyyyyy', 'caaaaaall', 'caaaaaall'

In [ ]:
Consec_4_wr_letter_words = []
for word in Consec_4_letter_words:
  new_word = ''
  for idx, char in enumerate(word):
    if idx + 1 < len(word):
      
      if word[idx] != word[idx + 1]:
        # print(word[idx], word[idx + 1])
        new_word = new_word + char
    else:
      # if word[idx - 1] == word[idx]:
      new_word = new_word + char
  Consec_4_wr_letter_words.append(new_word)
        
print(len(Consec_4_wr_letter_words))
print(Consec_4_wr_letter_words)

84
['so', 'aw', 'alexship', 'back', 'xdoj', 'gr', 'z', 'lmfao', 'snif', 'snif', 'snif', 'uh', 'uh', 'loveyou', 'kwadead', 'kwadead', 'pusy', 'pusy', 'pusy', 'pusy', 'pusy', 'pusy', 'ah', 'ah', 'ohyayay', 'al', 'al', 'al', 'al', 'al', 'snel', 'i', 'river', 'river', 'river', 'milio', 'romsgr', 'al', 'al', 'now', 'now', 'now', 'womp', 'selmo', 'selmo', 'selmo', 'er', 'damn', 'and', 'shid', 'ah', 'omg', 'a', 'on', 'come', 'come', 'mxa', 'l', 'bla', 'bla', 'bla', 'bla', 'ayhdjfjrjrdjeks', 'ayhdjfjrjrdjeks', 'mochichi', 'mochichi', 'shetingand', 'shetingand', 'shetingand', 'man', 'man', 'rudy', 'rudy', 'rudy', 'cal', 'cal', 'cal', 'wo', 'wo', 'wo', 'wo', 'rios', 'rios', 'nice']


In [ ]:
from collections import Counter

new_words = Counter(Consec_4_wr_letter_words)

for j, i in new_words.items():
  print(j, i)

so 1
aw 1
alexship 1
back 1
xdoj 1
gr 1
z 1
lmfao 1
snif 3
uh 2
loveyou 1
kwadead 2
pusy 6
ah 3
ohyayay 1
al 7
snel 1
i 1
river 3
milio 1
romsgr 1
now 3
womp 1
selmo 3
er 1
damn 1
and 1
shid 1
omg 1
a 1
on 1
come 2
mxa 1
l 1
bla 4
ayhdjfjrjrdjeks 2
mochichi 2
shetingand 3
man 2
rudy 3
cal 3
wo 4
rios 2
nice 1


#Sub program list of chat language

In [ ]:
word_corpus = []
j = 0
for tweet in Tweets_PreProcessed['tokenized_processed']:
  for word in tweet:
    if word in word_corpus:
      j = j +1
    else:
      word_corpus.append(word)
print(len(word_corpus))

12866


In [ ]:
two_letter_words = []

for word in word_corpus:
  if len(word) == 2:
    two_letter_words.append(word)
print((two_letter_words))

['of', 'us', 'la', 'to', 'in', 'by', 'no', 'or', 'as', 'am', 'on', 'an', 'is', 'so', 'do', 'we', 'up', 'my', 'rt', 'me', 'at', 'it', 'ûò', 'ûó', 'st', 'ig', 'fb', 'be', 'if', 'wm', 'en', 'go', 'nb', 'rd', 'pm', 'av', 'sb', 'sr', 'dr', 'th', 'nc', 'ps', 'ir', 'xb', 'tc', 'bb', 'eu', 'hu', 'cr', 'ca', 'ok', 'ny', 'he', 'ûï', 'em', 'un', 'bc', 'lg', 'vs', 'az', 'oh', 'vi', 'mo', 'xv', 'gt', 'xp', 'cd', 'el', 'ww', 'pd', 'eh', 'tv', 'ki', 'ìñ', 'da', 'mc', 'bk', 'ii', 'pt', 'ik', 'xl', 'px', 'lt', 'yo', 'wd', 'nd', 'ya', 'tb', 'du', 'dc', 'id', 'ft', 'gm', 'll', 'gh', 'dj', 'bu', 'ms', 'fm', 'rn', 'gf', 'ch', 'pl', 're', 'mb', 'te', 'hw', 'tf', 'ty', 'rq', 'ye', 'au', 'dq', 'hi', 'lb', 'pc', 'sg', 'ah', 've', 'bp', 'se', 'bs', 'ga', 'rs', 'ur', 'ee', 'ra', 'al', 'ìü', 'jp', 'br', 'ep', 'po', 'es', 'va', 'js', 'mf', 'yr', 'wr', 'ri', 'mp', 'ed', 'et', 'co', 'nh', 'uh', 'fr', 'vc', 'rl', 'er', 'pp', 'mv', 'bg', 'ov', 'åç', 'åè', 'sf', 'nu', 'md', 'ha', 'ââ', 'sa', 'sl', 'wi', 'bd', 'ks', 'åê

#Other features to be tried

In [ ]:
# To remove leading and ending spaces

text = text.strip()